In [271]:
#STREAMS2TABLE
#written by Zachary J.J. Roper
#z.roper@vanderbilt.edu, zach@roper.io

#A script for converting raw event codes and timestamps into a trial-item dataset

In [272]:
#This notebook requires the following modules
import os
import numpy
import csv

In [273]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [274]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [275]:
events.count('2756') #catch trials

9

In [276]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [277]:
#uberlist

In [278]:
'2810' in uberlist[74][2]

False

In [279]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [280]:
#RT

In [281]:
#The following provides indexed lists for each of the individual event codes of interest 

In [282]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [283]:
#float(times[74])-float(times[70])

In [284]:
#Saccade

In [285]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [286]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [287]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [288]:
#print(TimeTrials[0])

In [289]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [290]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
tloc = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0
    #if '5000' in trial:
    #    tloc[i] = 360
    if '5045' in trial:
        tloc[i] = 45
    elif '5090' in trial:
        tloc[i] = 90
    elif '5135' in trial:
        tloc[i] = 135
    elif '5180' in trial:
        tloc[i] = 180
    elif '5225' in trial:
        tloc[i] = 225
    elif '5270' in trial:
        tloc[i] = 270
    elif '5315' in trial:
        tloc[i] = 315
    else:
        tloc[i] = 0

In [291]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [292]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [293]:
#RT

In [294]:
checksum = sum(error)+sum(abort)+sum(catch)

In [295]:
checksum

215

In [296]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i],tloc[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT', 'TarLoc']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT,tloc))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 1 0 0 0 0 0 180
2 0 1 0 1 0 221.96219999999994 90
3 1 0 0 0 0 0 315
4 0 1 0 1 0 183.99239999999918 270
5 0 0 1 0 0 163.96290000000045 45
6 0 0 0 1 0 181.9854000000014 180
7 1 0 0 0 0 0 270
8 0 1 0 1 0 215.98219999999856 180
9 0 1 0 1 0 198.00069999999687 225
10 0 1 0 1 0 178.01209999999992 270
11 0 1 0 1 0 178.01239999999962 0
12 0 1 0 1 0 227.9832000000024 0
13 0 1 0 1 0 158.02360000000044 0
14 0 1 0 1 0 203.98070000000007 135
15 0 1 0 1 0 237.9777000000031 90
16 0 1 0 1 0 231.99749999999767 90
17 0 1 0 1 0 215.98220000000583 180
18 1 0 0 0 0 0 180
19 0 1 0 1 0 159.9896000000008 0
20 0 1 0 1 0 234.00450000000274 315
21 0 1 0 1 0 167.97690000000148 45
22 1 0 0 0 0 0 45
23 0 1 0 1 0 179.97810000000027 135
24 0 1 0 1 0 155.97559999999794 315
25 0 1 0 1 0 172.03190000000177 225
26 0 1 0 1 0 171.99109999999928 90
27 0 1 0 1 0 225.97619999999006 90
28 0 1 0 1 0 169.9838000000018 45
29 0 1 0 1 0 165.96980000000622 315
30 1 0 0 0 0 0 0
31 0 1 0 1 0 173.99790000000212 315
32 0 1 0 1 0 222.00

531 0 1 0 1 0 193.98640000005253 315
532 0 1 0 1 0 258.0070999999298 180
533 0 1 0 1 0 183.99210000003222 135
534 0 1 0 1 0 181.98519999999553 90
535 0 1 0 1 0 191.93859999999404 90
536 0 1 0 1 0 198.000400000019 0
537 1 0 0 0 0 0 135
538 1 0 0 0 0 0 90
539 0 1 0 1 0 199.96669999998994 90
540 0 1 0 1 0 181.985100000049 0
541 0 1 0 1 0 167.97709999990184 180
542 0 1 0 1 0 199.96689999999944 90
543 0 1 0 1 0 173.99809999996796 180
544 0 1 0 1 0 211.96799999999348 225
545 0 1 0 1 0 217.9892000000691 315
546 0 1 0 1 0 260.01410000002943 225
547 0 1 0 1 0 225.97649999998976 135
548 0 1 0 1 0 197.95960000006016 225
549 0 1 0 1 0 217.9889999999432 180
550 1 0 0 0 0 0 270
551 1 0 0 0 0 0 180
552 0 1 0 1 0 229.99040000000969 270
553 1 0 0 0 0 0 315
554 1 0 0 0 0 0 270
555 0 1 0 1 0 235.9704999999376 135
556 0 1 0 1 0 223.96929999999702 45
557 0 1 0 1 0 193.98660000006203 135
558 0 1 0 1 0 211.96799999999348 315
559 0 1 0 1 0 249.97879999992438 315
560 0 1 0 1 0 242.03280000004452 90
561 0 1 0 1

In [297]:
sum(catch)

9